# Interactive lightcurves alongside animated target pixel files

One of the most common questions we get at the Kepler/K2 Guest Observer Office is "I see a noise blip in my K2 lightcurve.  Is it real or instrumental?"  Answering the question usually entails ticking through our (long, incomplete) mental list of conceivable, sometimes exotic instrumental artifacts.  Space-craft induced periodic motion.  Rolling band.  Cosmic Rays.  Collateral cosmic rays.  Smear.  Sudden pixel sensitivity dropouts.  Fine-guidance-sensor cross talk.  Regular cross talk. [Sweater fuzzy](https://twitter.com/gully_/status/941707161058586624).

Sometimes the ephemeral signals arise from exotic astrophysics, but not the kind that interests our *K2-user-in-need*.  Asteroid conjunctions, Mars' bleed trail, background eclipsing binaries, and crowded fields all conspire to induce seemingly-real astrophysical signals.  **Many effects matter at the exceptional precision level of Kepler and K2.**

No single algorithm can distinguish the wheat from the chaff in Kepler lightcurves, though heuristics exist.  [K2Flix](https://github.com/KeplerGO/k2flix) provides user-friendly animated gifs that satisfy the inner Feynman- "You just look at the thing!".  The sane defaults for screen stretch mean that flares and rolling-band alike appear conspicuously in the [tweet-ready gifs](https://twitter.com/keplerbot?lang=en).

<img src=https://raw.githubusercontent.com/barentsen/k2flix/master/examples/epic-201572338.gif></img>

One missing ingredient evades K2Flix. Registering the fateful, flawed moment to the position in a lightcurve requires manual intervention by spot-checking a lookup-table of time, or cadence, or some other bespoke tool to snip out the irrecoverable data.  *I want to see the effect of the artifact on the lightcurve in real time.*  K2 Guest Observer Director Geert Barentsen has, for a long time, sought a tool that shows the K2 lightcurve with a slider bar that continually updates the TPF snapshot registered at the user-selected cadence.  

In this tutorial we introduce and explain the `tpf.interact()` tool that accomplishes these goals.

### Additional installation required

The `tpf.interact()` tool requires two Python dependencies that are not required anywhere else in the lightkurve codebase: [bokeh](https://bokeh.pydata.org/en/latest/) and [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).  These dependencies can be installed effortlessly with anaconda:

```bash
conda install bokeh ipywidgets
```

The bokeh Application Programming Interface (API) tends to change regularly with its rapid pace of development, so it is conceivable and likely that you will need to install the same or similiar-enough version of `bokeh` as we have used in its developement and evaluation.  So far we have tested `.inteact()` with version 0.12.15 of `bokeh`.  You can specificy this specific version with the following modification to the above commandline tool:


```bash
conda install bokeh=0.12.15 ipywidgets
```

You'll also need a modern web-browser and Jupyter Notebooks.  JupyterLab is not yet supported.   

You may be unable or reluctant to install/manage these extra dependencies.  Rest assured, all of the rest of `lightkurve` tools will still work without these dependencies, you simply will not be able to enjoy the interactive interface laid out in this tutorial.  Attempts to run `.interact()` without these dependencies will be met with a graceful `ImportError`.

### Using `.interact()`

Using interact should be as simple as downloading a Kepler Target Pixel File (TPF) and running the method `.interact()`.  This method can only be run in a Jupyter Notebook at the moment.   

Let's use the exact same TPF as the K2Flix gif demo from above EPIC 201572338, which comes from Campaign 1.  It's animated gif shows some conspicuous examples of asteroid conjunctions.

In [21]:
from lightkurve import KeplerTargetPixelFile
tpf = KeplerTargetPixelFile.from_archive(201572338)

The default lightcurves comes from the default "pipeline" mask, typically only a few pixels centered on the target photocenter, depending on factors like the target brightness and its expected PSF size.  

In [22]:
tpf.interact()

Loading BokehJS ...

A Jupyter Widget

A Jupyter Widget

You can move the large bottom left slider to change the location of the verical red bar, which indicates which cadence is being shown in the TPF postage stamp image.  You can also hit the play button to automatically play-though all the cadences.  The left-and-right arrow keys on your keyboard will tick through the cadences one-by-one, if the widget slider is active (i.e. it's the most recent thing you clicked on).  The slider beneath the TPF postage stamp image controls the screen stretch, which defaults to logarithmic scaling initialized to 1% and 95% lower and upper limits respectively.

You can move your cursor over individual data points to show hover-over tool-tips indicating additional information about that datum.  Currently the tool tips list the cadence, time, flux, and quality flags.  The tools on the right hand side of the plots enable zooming, panning, and toggling-off tool-tips.

### Passing in a preprocessed lightcurve

You may wish to visualize a custom-processed lightcurve adjacent to its TPF postage stamp representation.  You can pass-in a lightcurve as an optional keyword argument like so:

In [23]:
import numpy as np

In [24]:
def my_custom_lc_cleaner(tpf, cleaning='best'):
    '''Returns a clean lightcurve given an input tpf'''
    
    nan_mask = np.all(np.isnan(tpf.flux), axis=0)
    empty_mask = tpf.pipeline_mask * False
    aper_mask = empty_mask
    aper_mask[1:-1, 1:-1] = True
    back_mask = ~aper_mask & ~nan_mask
    
    lc_aper = tpf.to_lightcurve(aperture_mask=aper_mask)/aper_mask.sum()
    lc_back = tpf.to_lightcurve(aperture_mask=back_mask)/back_mask.sum()
    
    lc_bg_sub = lc_aper - lc_back.flux
    
    # Flag outliers
    _, outliers1 = lc_aper.remove_outliers(return_mask=True)
    _, outliers2 = lc_back.remove_outliers(return_mask=True)
    _, outliers3 = lc_aper.flatten().remove_outliers(return_mask=True)

    outlier_dict= {'none': outliers1*False,
                   'safe':(tpf.quality > 0.0),
                   'soft': outliers1,
                   'medium':(outliers1 | outliers2),
                   'best': (outliers1 | outliers2 | (tpf.quality > 0.0)),
                   'aggressive':(outliers1 | outliers2 | (tpf.quality > 0.0) | outliers3)}

    outliers = outlier_dict[cleaning]
    
    lc_cln = lc_bg_sub[~outliers]
    
    return lc_cln

In [27]:
lc_cln = my_custom_lc_cleaner(tpf)

In [28]:
tpf.interact(lc=lc_cln)

Loading BokehJS ...

A Jupyter Widget

A Jupyter Widget

Your clean lightcurve may have a different number of cadences than your target pixel file, for example if you dropped outliers.  The TPF will show all of the available cadences by default, with missing spaces in the lightcurve. The vertical red bar will indicate where-in-time the cadences would have appeared.

We hope you enjoy the `.interact()` tool!  Requests for extensions or more features are welcomed on our [GitHub Issues](https://github.com/KeplerGO/lightkurve/issues).